In [1]:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import re
from keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from keras.layers import Conv2D,MaxPooling2D,\
     Dropout,Flatten,Dense,Activation,\
     BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
import pandas as pd

In [2]:
folder_path = r'D:\Studying\ApplAi_CV\Assignments\second\train'
categories = ['horses' , 'humans']
testdirr = r'D:\Studying\ApplAi_CV\Assignments\second\test'

In [3]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [4]:
filenames = os.listdir(folder_path)
data = []

for name in filenames:
    category_path = os.path.join(folder_path, name)
    if os.path.isdir(category_path):
        category_label = name
        for sub in os.listdir(category_path):
            data.append({'filename': os.path.join(category_path,sub), 'label': category_label})
            
df = pd.DataFrame(data)

In [5]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
train_gen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=0.4,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_gen.flow_from_dataframe(
    train_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1.0 / 255)

val_generator = val_gen.flow_from_dataframe(
    val_df,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

Found 960 validated image filenames belonging to 2 classes.
Found 240 validated image filenames belonging to 2 classes.


In [6]:
cnn_model = Sequential([
    Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPool2D((2,2)),

    Conv2D(32, (3,3), activation='relu'),
    MaxPool2D((2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D((2,2)),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPool2D((2,2)),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])



In [8]:
cnn_model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = cnn_model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator,
)

Epoch 1/15
30/30 [==============================] - 12s 383ms/step - loss: 0.6852 - accuracy: 0.5833 - val_loss: 0.6233 - val_accuracy: 0.7625
Epoch 2/15
30/30 [==============================] - 11s 372ms/step - loss: 0.6581 - accuracy: 0.6135 - val_loss: 0.6240 - val_accuracy: 0.6417
Epoch 3/15
30/30 [==============================] - 11s 368ms/step - loss: 0.5676 - accuracy: 0.7208 - val_loss: 0.5212 - val_accuracy: 0.8042
Epoch 4/15
30/30 [==============================] - 11s 363ms/step - loss: 0.4681 - accuracy: 0.7823 - val_loss: 0.3428 - val_accuracy: 0.8458
Epoch 5/15
30/30 [==============================] - 11s 362ms/step - loss: 0.3974 - accuracy: 0.8479 - val_loss: 0.2981 - val_accuracy: 0.8958
Epoch 6/15
30/30 [==============================] - 11s 360ms/step - loss: 0.3660 - accuracy: 0.8406 - val_loss: 0.2394 - val_accuracy: 0.9292
Epoch 7/15
30/30 [==============================] - 11s 363ms/step - loss: 0.3010 - accuracy: 0.8823 - val_loss: 0.1888 - val_accuracy: 0.9417

In [9]:
img_size = 150
testData = []
filaty = []  

image_files = [imagee for imagee in os.listdir(testdirr)]

sorted_image_files = sorted(image_files, key=lambda x: int(re.search(r'\d+', x).group()))

for imagee in sorted_image_files:
    image_path = os.path.join(testdirr, imagee)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_size, img_size))
    testData.append(image)
    
    filaty.append(imagee)

X_test = np.array(testData)
X_test = X_test/255

In [11]:
predictions = cnn_model.predict(X_test)

10/10 [==============================] - 0s 31ms/step


In [12]:
predicted_labels = [1 if prediction[0] >= 0.5 else 0 for prediction in predictions]


In [13]:
predicted_labels

[1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [14]:
import csv
csv_filename = "Submission_file.csv"  

label_map = {0: "horse", 1: "human"}

with open(csv_filename, mode="r") as csv_file:
    rows = list(csv.reader(csv_file))

for i, row in enumerate(rows[1:]):  
    prediction_label = label_map[int(predicted_labels[i])]
    row[1] = prediction_label

for i in range(len(rows)):
    if i > 0: 
        rows[i][0] = filaty[i - 1] 
        
with open(csv_filename, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(rows)